In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\janha\\Health-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\janha\\Health-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File 

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                         glob="*.pdf",
                         loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [11]:
#text_chunks

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
#pip install sentence-transformers

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\janha\AppData\Local\Temp\ipykernel_22000\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\janha\miniconda3\envs\healthbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [18]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_UWfTo_8FyHHoCj6NMrkyYrctTkc8QG8KCxAM5ofAojoRUwadDPTkTgL8uenKcuAJncteU")
index_name = "testbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '6174c4491ee85f9d8688db4810835005', 'Date': 'Thu, 16 Jan 2025 06:47:14 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [21]:
import os
os.environ["PINECONE_API_KEY"]="pcsk_UWfTo_8FyHHoCj6NMrkyYrctTkc8QG8KCxAM5ofAojoRUwadDPTkTgL8uenKcuAJncteU"


In [22]:
#Embed each chunk and upsert the embeddings into your Pinecone index. 
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


KeyboardInterrupt: 

In [23]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

#Embed each chunk and upsert the embeddings into your Pinecone index.

docsearch=PineconeVectorStore.from_existing_index(

    index_name=index_name,

    embedding=embeddings
)

In [24]:
docsearch

In [25]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [26]:
retrieved_docs=retriever.invoke("What is Acne?")

In [27]:
retrieved_docs

[Document(id='ac8e4e4d-d442-4281-a454-26037cbb3608', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='5f648825-d11e-4b1d-a491-22c6f4b07036', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='44ab6a8e-bfa8-4c38-8b18-5c349bb5422a', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]

In [28]:
from transformers import pipeline

# Initialize Hugging Face's text generation model
llm_pipeline = pipeline("text-generation", model="google/flan-t5-base")

# Function to generate a response
def generate_response(prompt):
    response = llm_pipeline(prompt, max_length=200, do_sample=True)
    return response[0]['generated_text']

# Example query
response = generate_response("Explain Acne in simple terms.")
print(response)

c:\Users\janha\miniconda3\envs\healthbot\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\janha\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported

Explain Acne in simple terms.remove natural scars, flaws, or lesions.


In [30]:
# Retrieve similar documents
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("What is Acne?")
print(retrieved_docs)

# Generate response using Hugging Face model
response = generate_response("What is acne?")
print(response)

[Document(id='ac8e4e4d-d442-4281-a454-26037cbb3608', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(id='5f648825-d11e-4b1d-a491-22c6f4b07036', metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'), Document(id='696564d9-6b3e-441b-a1ba-5df03376ae6c', metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]
What is acne?rts on your forehead


In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(extracted_data):
    # Increase chunk size and reduce overlap to capture full context
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    
    # Filter out non-informative chunks (like page numbers)
    cleaned_chunks = []
    for chunk in text_splitter.split_documents(extracted_data):
        if "GALE ENCYCLOPEDIA" not in chunk.page_content and "Page" not in chunk.page_content:
            cleaned_chunks.append(chunk)
    
    return cleaned_chunks

text_chunks = text_split(extracted_data)
print("Filtered chunks:", len(text_chunks))


Filtered chunks: 2399


In [35]:
retriever = docsearch.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10})

# Test the retrieval again
retrieved_docs = retriever.invoke("Explain what acne is and its symptoms.")
for doc in retrieved_docs:
    print(doc.page_content)


GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26
KEY TERMS
Acne—A skin condition in which raised bumps,
pimples, and cysts form on the face, neck, shoul-
ders and upper back.
Bacteria—Tiny, one-celled forms of life that cause
many diseases and infections.
Bowel—The intestine; a tube-like structure that
extends from the stomach to the anus. Some diges-
tive processes are carried out in the bowel before
food passes out of the body as waste.
Cyst—An abnormal sac or enclosed cavity in the
body, filled with liquid or partially solid material.
shoulders, and back since these are the parts of the body
where the most sebaceous follicles are found.
Causes and symptoms
The exact cause of acne is unknown. Several risk
factors have been identified:
• Age. Due to the hormonal changes they experience,
teenagers are more likely to develop acne.
• Gender. Boys have more severe acne and develop it
more often than girls.
• Disease. Hormonal disorders can complicat